In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
from torchvision.transforms.functional import pad as PAD
import torch.optim as optim
from torch.utils.data.dataset import random_split
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
import re
import pandas as pd
from sklearn.metrics import classification_report
import wandb
import numbers
import natsort
from PIL import Image
from torchvision.transforms import ToPILImage,ToTensor
ToPILImage = ToPILImage()
os.environ['CUDA_VISIBLE_DEVICES']='1'

from efficientnet_pytorch import EfficientNet

PyTorch Version:  1.4.0


In [2]:
hyperparameter_defaults = dict(
    input_size = 1024,
    batch_size = 1,
)
os.environ['WANDB_NOTEBOOK_NAME']='GCAM Upload Notebook'
wandb.init(project="GCAM Results",config=hyperparameter_defaults)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: szho6430 (use `wandb login --relogin` to force relogin)


In [3]:
from torch.utils.data import Dataset

class TestSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        img_name = self.total_imgs[idx]
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image,img_name

In [4]:
test_transform = transforms.Compose([
        transforms.Resize(1024),
        transforms.ToTensor()
    ])

In [5]:
GradCam_Sets = {x: TestSet(os.path.join('./grad-cam-pytorch/outputs/',x),transform=test_transform) for x in ['Direct','MURA','MURA2'] }

In [6]:
CamLoaders = {x:torch.utils.data.DataLoader(GradCam_Sets[x],batch_size=1,shuffle=False) for x in ['Direct','MURA','MURA2']}

In [7]:
LIST=pd.read_csv('./Post-Meta-Dz-Consolidated.csv',dtype={'Image No.':str})
LIST.head()

,Patient No.,Species,Breed,Age (yrs),Gender,Image Findings,Thoracic or Pelvic,Image No.,Is_Normal,Test_CV,Direct,MURA
0,1,Canine,German Shepherd,2,Female,Osteoarthrosis,Thoracic,001,False,4,1,1
1,1,Canine,German Shepherd,2,Female,Osteoarthrosis,Thoracic,002,False,1,1,1
2,1,Canine,German Shepherd,2,Female,Osteoarthrosis,Thoracic,003,False,3,0,0
3,1,Canine,German Shepherd,2,Female,Osteoarthrosis,Thoracic,004,False,3,1,1
4,1,Canine,German Shepherd,2,Female,Osteoarthrosis,Thoracic,005,False,4,1,1


In [10]:
for strategy in [
#     'Direct',
    'MURA',
]:
    for inputs,name in CamLoaders['MURA2']:
        inputs = ToPILImage(inputs.squeeze())
        name = name[0].split('-')[1]
        image_no = name.replace('.jpeg','').replace('.jpg','')
        correctness = LIST.loc[LIST['Image No.']==image_no,strategy].item()
        normal = LIST.loc[LIST['Image No.']==image_no,'Is_Normal'].item()
        if normal:
            if correctness == 1:
                caption = strategy+'_Finetune\nPrediction\tNormal'
            if correctness == 0:
                caption = strategy+'_Finetune\nPrediction\tAbnormal'
        if normal==False:
            if correctness == 1:
                caption = strategy+'_Finetune\nPrediction\tAbnormal'
            if correctness == 0:
                caption = strategy+'_Finetune\nPrediction\tNormal'
        wandb.log({image_no:[wandb.Image(inputs,caption=caption)]})
        print(f"{image_no}\t{caption}")

/home/dasleo/miniconda3/envs/pytorch13/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: `item` has been deprecated and will be removed in a future version
  if __name__ == '__main__':
/home/dasleo/miniconda3/envs/pytorch13/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


001	MURA_Finetune
Prediction	Abnormal
002	MURA_Finetune
Prediction	Abnormal
003	MURA_Finetune
Prediction	Normal
004	MURA_Finetune
Prediction	Abnormal
005	MURA_Finetune
Prediction	Abnormal
006	MURA_Finetune
Prediction	Abnormal
007	MURA_Finetune
Prediction	Abnormal
008	MURA_Finetune
Prediction	Abnormal
009	MURA_Finetune
Prediction	Abnormal
010	MURA_Finetune
Prediction	Abnormal
011	MURA_Finetune
Prediction	Normal
012	MURA_Finetune
Prediction	Normal
013	MURA_Finetune
Prediction	Abnormal
014	MURA_Finetune
Prediction	Abnormal
015	MURA_Finetune
Prediction	Abnormal
016	MURA_Finetune
Prediction	Abnormal
017	MURA_Finetune
Prediction	Normal
018	MURA_Finetune
Prediction	Abnormal
019	MURA_Finetune
Prediction	Normal
020	MURA_Finetune
Prediction	Abnormal
021	MURA_Finetune
Prediction	Abnormal
022	MURA_Finetune
Prediction	Abnormal
023	MURA_Finetune
Prediction	Abnormal
024	MURA_Finetune
Prediction	Abnormal
025	MURA_Finetune
Prediction	Abnormal
026	MURA_Finetune
Prediction	Abnormal
027	MURA_Finetune
Pred

219	MURA_Finetune
Prediction	Abnormal
220	MURA_Finetune
Prediction	Abnormal
221	MURA_Finetune
Prediction	Abnormal
222	MURA_Finetune
Prediction	Abnormal
223	MURA_Finetune
Prediction	Abnormal
224	MURA_Finetune
Prediction	Abnormal
225	MURA_Finetune
Prediction	Abnormal
226	MURA_Finetune
Prediction	Abnormal
227	MURA_Finetune
Prediction	Abnormal
228	MURA_Finetune
Prediction	Abnormal
229	MURA_Finetune
Prediction	Abnormal
230	MURA_Finetune
Prediction	Abnormal
231	MURA_Finetune
Prediction	Abnormal
232	MURA_Finetune
Prediction	Abnormal
233	MURA_Finetune
Prediction	Abnormal
234	MURA_Finetune
Prediction	Abnormal
235	MURA_Finetune
Prediction	Abnormal
236	MURA_Finetune
Prediction	Abnormal
237	MURA_Finetune
Prediction	Normal
238	MURA_Finetune
Prediction	Normal
239	MURA_Finetune
Prediction	Normal
240	MURA_Finetune
Prediction	Normal
241	MURA_Finetune
Prediction	Normal
242	MURA_Finetune
Prediction	Normal
243	MURA_Finetune
Prediction	Normal
244	MURA_Finetune
Prediction	Abnormal
245	MURA_Finetune
Predicti

440	MURA_Finetune
Prediction	Abnormal
441	MURA_Finetune
Prediction	Abnormal
442	MURA_Finetune
Prediction	Abnormal
443	MURA_Finetune
Prediction	Abnormal
444	MURA_Finetune
Prediction	Abnormal
445	MURA_Finetune
Prediction	Normal
446	MURA_Finetune
Prediction	Normal
447	MURA_Finetune
Prediction	Normal
448	MURA_Finetune
Prediction	Abnormal
449	MURA_Finetune
Prediction	Normal
450	MURA_Finetune
Prediction	Normal
451	MURA_Finetune
Prediction	Abnormal
452	MURA_Finetune
Prediction	Abnormal
453	MURA_Finetune
Prediction	Abnormal
454	MURA_Finetune
Prediction	Abnormal
455	MURA_Finetune
Prediction	Abnormal
456	MURA_Finetune
Prediction	Normal
457	MURA_Finetune
Prediction	Normal
458	MURA_Finetune
Prediction	Abnormal
459	MURA_Finetune
Prediction	Normal
460	MURA_Finetune
Prediction	Normal
461	MURA_Finetune
Prediction	Normal
462	MURA_Finetune
Prediction	Abnormal
463	MURA_Finetune
Prediction	Abnormal
464	MURA_Finetune
Prediction	Abnormal
465	MURA_Finetune
Prediction	Normal
466	MURA_Finetune
Prediction	Norma